In [1]:
import sys
sys.path.insert(0, '..')
from utils.preprocesamiento import StemmerTokenizer

tokenizador = StemmerTokenizer(stem=False,rmv_punctuation=True)

In [2]:
from utils.cargar import df_caso
from utils.preprocesamiento import process_df

caso = 'alicia'
df = df_caso(caso)

df = process_df(df,'comment','sel',verbose=True)

df = df.drop(columns=['user_id','team_id','gender','df','title','opt_left','opt_right','max_num','phase','time','curso'])

80 rows found with non string elements for column comment (0.65%)
Deleting 2326 columns for which max target value is over 7 (18.76%)
9991 available rows after processing


In [3]:
from sklearn.model_selection import train_test_split

df_train, df_test, _, _ = train_test_split(df, df['sel'], test_size=.05, stratify=df['sel'], random_state=0)

In [4]:
tokenized_corpus = [tokenizador(document) for document in df_train['comment']]
tokenized_test = [tokenizador(document) for document in df_test['comment']]

In [5]:
import gensim
from gensim import corpora

# Create a dictionary from the tokenized corpus
dictionary = corpora.Dictionary(tokenized_corpus)

# Convert the tokenized corpus into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_corpus]

# Perform topic modeling using LDA
num_topics = 2  # Number of topics to generate
lda_model = gensim.models.LdaModel(doc_term_matrix, num_topics=num_topics, id2word=dictionary, passes=10)

In [6]:
# Print the generated topics
topics = lda_model.print_topics(num_topics=num_topics)
for topic in topics:
    print(topic)

(0, '0.041*"proyecto" + 0.019*"entregar" + 0.016*"plazos" + 0.016*"criterios" + 0.015*"cumplir" + 0.014*"importante" + 0.013*"plazo" + 0.013*"priorizar" + 0.012*"debe" + 0.012*"técnicos"')
(1, '0.022*"proyecto" + 0.017*"ser" + 0.016*"si" + 0.013*"reputación" + 0.012*"Alicia" + 0.011*"puede" + 0.010*"usuarios" + 0.010*"contexto" + 0.010*"transparencia" + 0.008*"debe"')


In [7]:
df_test.head()

,sel,comment
id,,
36695,2,Es importante producir un producto aceptable i...
23603,4,Si los plazos no son flexibles y esto escapa d...
28106,4,Creo que en primer caso podría ser honesta y c...
33310,3,Alicia debería intentar terminar el proyecto e...
32310,1,Debido a que no cumpliría con los lineamientos...


In [8]:
tokenized_test[0]

['Es',
 'importante',
 'producir',
 'producto',
 'aceptable',
 'incluso',
 'si',
 'demora',
 'pues',
 'hacer',
 'producto',
 'mal',
 'puede',
 'llevar',
 'repercuciones',
 'grandes']

In [9]:
# Convert the tokenized document into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(tokenized_test[0])]

# Get the topic probabilities for the new document
topic_probs = lda_model.get_document_topics(doc_term_matrix)[0]

# Print the topic probabilities
for topic, prob in topic_probs:
    print(f"Topic {topic}: {prob}")

Topic 0: 0.5952473282814026
Topic 1: 0.404752641916275
